In [1]:
pwd

'/Users/robincole/Documents/Github/HASS-folder-watcher/development'

In [2]:
import datetime
import os

In [4]:
path = 'dummy_folder'

In [3]:
def get_timestamp(dir_entry):
    """Return the timestamp of file modification."""
    mtime = dir_entry.stat().st_mtime
    return datetime.datetime.fromtimestamp(mtime).isoformat()

In [14]:
def get_files(path):
    """Return the dict of files and timestamps."""
    files = {}
    with os.scandir(path) as it:
        for entry in it:
            if not entry.name.startswith('.') and entry.is_file():
                files[entry.name]= get_timestamp(entry)
    return files

In [5]:
files = {}

The following is the update method

In [18]:
# Original
current_files = [] 
deleted_files = []
added_files = []
modified_files = []
previous_files = list(files.keys())

with os.scandir(path) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_file():
            file_name = entry.name
            current_files.append(file_name)  # Keep record of current files
            file_mtime = get_timestamp(entry)
            
            # Logic to process files
            if not file_name in files:
                added_files.append(file_name)
                # print("{} added".format(file_name))
                files[file_name] = file_mtime # Add the entry
                
            elif file_name in files and files[file_name] != file_mtime: # If exists and modified
                modified_files.append(file_name)
                # print("{} modified".format(file_name))
                files[file_name] = file_mtime # Reassign
                

# Check if any files deleted                
deleted_files = list(set(previous_files) - set(current_files))
for file_name in deleted_files:
    # print("{} deleted".format(file_name))
    files.pop(file_name, None)

In [21]:
original_files = get_files(path)
original_files

{'file1.txt': '2018-02-28T09:05:14.348702',
 'file2.txt': '2018-02-28T09:15:36.982517'}

In [23]:
list(original_files.keys())

['file2.txt', 'file1.txt']

## Class implementation

In [33]:
"""The dict self._files is updated and events published on changes to the dict."""

class Watcher():
    def __init__(self, path):
        self._path = path
        self._updated = False
        self._files = get_files(self._path)

    def update(self):
        """Update the watcher."""
        current_files = []
        deleted_files = []
        added_files = []
        modified_files = []
        previous_files = list(self._files.keys())
        
        with os.scandir(self._path) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file(): 
                    file_mtime = get_timestamp(entry)
                    file_name = entry.name
                    current_files.append(file_name)  # Keep record of current files   
            
                    if not file_name in self._files:
                        added_files.append(file_name)
                        print("{} added".format(file_name))
                        self._files[file_name] = file_mtime # Add the entry
                
                    elif file_name in self._files and self._files[file_name] != file_mtime: # If exists and modified
                        modified_files.append(file_name)
                        print("{} modified".format(file_name))
                        self._files[file_name] = file_mtime # Update timestamp

            # Check if any files deleted                
            deleted_files = list(set(previous_files) - set(current_files))
            for file_name in deleted_files:
                print("{} deleted".format(file_name))
                self._files.pop(file_name, None) # Delete the entry

In [34]:
my_watcher = Watcher(path)

In [35]:
my_watcher.update()

In [36]:
my_watcher.update()

file2.txt modified


In [37]:
my_watcher.update()

file1 copy.txt added


In [38]:
my_watcher.update()

file1 copy.txt deleted
